In [1]:
%cd ..
import gym
import gym_game
import matplotlib.pyplot as plt
env = gym.make('Parkme')
reset, _ = env.reset()
n_actions = env.action_space.n
states_dim = env.observation_space.shape[0]
print(n_actions, states_dim)

/home/yvovaa/fork/HSE_ML_P2024
9 7


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [2]:
import torch.nn as nn
from torch.optim import Adam
hidden_dim = 128
model = nn.Sequential(
    nn.Linear(states_dim, hidden_dim),
    nn.Tanh(),
    nn.Linear(hidden_dim, n_actions),
    nn.Softmax()
)
optimizer = Adam(model.parameters())

In [3]:
import numpy as np
import random
import torch
def generate_session(env, agent, exploration_rate=0.8):
    states, actions = [], []
    total_reward = 0
    s, _ = env.reset()
    for _ in range(50000):
        if random.random() > exploration_rate:
            action = np.random.choice(list(range(n_actions)))
        else:
            with torch.no_grad():
                probs = agent(torch.FloatTensor(s))
                action = np.random.choice(list(range(n_actions)), p = probs.numpy())    
        new_s, r, terminated, truncated, _ = env.step(action)
        states.append(s)
        actions.append(action)
        total_reward += r

        s = new_s
        if terminated or truncated:
            break
    return states, actions, total_reward

In [4]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile):
    reward_threshold = np.percentile(rewards_batch, percentile)
    elite_states = []
    elite_actions = []
    for i in range(len(states_batch)):
        if rewards_batch[i] >= reward_threshold:
            elite_states.extend(states_batch[i])
            elite_actions.extend(actions_batch[i])

    return elite_states, elite_actions

In [5]:
%%time
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
criterion = CrossEntropyLoss()
percentile = 90
log = []
n_epochs = 300
best_reward = 0
for i in tqdm(range(n_epochs)):
    sessions = [generate_session(env, model) for _ in tqdm(range(100))]
    states_batch, actions_batch, rewards_batch = zip(*sessions)
    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)
    outps = model(torch.FloatTensor(elite_states))
    loss = criterion(outps.float(), torch.FloatTensor(elite_actions))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if np.mean(rewards_batch) > best_reward:
        best_reward = np.mean(rewards_batch)
    show_progress(
        rewards_batch, log, percentile, reward_range=[0, max(0, np.max(rewards_batch))]
    )

  0%|          | 0/300 [00:00<?, ?it/s]/home/yvovaa/.local/lib/python3.10/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
100%|██████████| 1/1 [00:00<00:00,  1.44it/s]

: 